### 分类模型的构建与训练


#### 数据集获取

MNIST数据集可在[http://yann.lecun.com/exdb/mnist/]() 直接下载，也可以使用Tensorflow提供的数据集读取方法获取。

In [7]:
import tensorflow as tf 
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets('MNIST/',one_hot=True)

ModuleNotFoundError: No module named 'tensorflow.examples.tutorials'

In [6]:
print(tf.__version__)

2.0.0


In [2]:
# 定义x和y的占位符
# mnist 中每张图片共有28*28=784个像素点
x = tf.placeholder(tf.float32, [None, 784], name="X") 

# 0-9 一共10个数字=> 10 个类别
y = tf.placeholder(tf.float32, [None, 10], name="Y")  

# 定义变量
W = tf.Variable(tf.random_normal([784, 10]),name="W")
b = tf.Variable(tf.zeros([10]),name="b") 

forward=tf.matmul(x, W) + b # 前向计算

pred = tf.nn.softmax(forward) # Softmax分类

In [3]:
# 设置训练参数
train_epochs = 50 # 训练轮数
batch_size = 100  # 单次训练样本数（批次大小）
total_batch= int(mnist.train.num_examples/batch_size)  # 一轮训练有多少批次
display_step = 1  # 显示粒度
learning_rate= 0.01  # 学习率

# 定义交叉熵损失函数
loss_function = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), 
                                              reduction_indices=1)) 
                                              
# 梯度下降优化器
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_function)

 # 定义准确率
 # 检查预测类别tf.argmax(pred, 1)与实际类别tf.argmax(y, 1)的匹配情况
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# 准确率，将布尔值转化为浮点数，并计算平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) 

sess = tf.Session() #声明会话
init = tf.global_variables_initializer() # 变量初始化
sess.run(init)

# 开始训练
for epoch in range(train_epochs ):
    for batch in range(total_batch):
        xs, ys = mnist.train.next_batch(batch_size)# 读取批次数据
        sess.run(optimizer,feed_dict={x: xs,y: ys}) # 执行批次训练
    
    #total_batch个批次训练完成后，使用验证数据计算误差与准确率；验证集没有分批   
    loss,acc = sess.run([loss_function,accuracy],
                        feed_dict={x: mnist.validation.images, y: mnist.validation.labels})
    # 打印训练过程中的详细信息
    if (epoch+1) % display_step == 0:
        print("Train Epoch:", '%02d' % (epoch+1), "Loss=", "{:.9f}".format(loss),\
              " Accuracy=","{:.4f}".format(acc))

print("Train Finished!")  


Train Epoch: 01 Loss= 5.579719543  Accuracy= 0.2880
Train Epoch: 02 Loss= 3.516610622  Accuracy= 0.4544
Train Epoch: 03 Loss= 2.631449223  Accuracy= 0.5532
Train Epoch: 04 Loss= 2.161327839  Accuracy= 0.6140
Train Epoch: 05 Loss= 1.868612647  Accuracy= 0.6548
Train Epoch: 06 Loss= 1.670349956  Accuracy= 0.6872
Train Epoch: 07 Loss= 1.526025772  Accuracy= 0.7110
Train Epoch: 08 Loss= 1.416240096  Accuracy= 0.7298
Train Epoch: 09 Loss= 1.328943014  Accuracy= 0.7440
Train Epoch: 10 Loss= 1.257907271  Accuracy= 0.7572
Train Epoch: 11 Loss= 1.199871659  Accuracy= 0.7682
Train Epoch: 12 Loss= 1.149572492  Accuracy= 0.7772
Train Epoch: 13 Loss= 1.107563257  Accuracy= 0.7852
Train Epoch: 14 Loss= 1.070685744  Accuracy= 0.7920
Train Epoch: 15 Loss= 1.038186908  Accuracy= 0.7982
Train Epoch: 16 Loss= 1.008350372  Accuracy= 0.8014
Train Epoch: 17 Loss= 0.981877327  Accuracy= 0.8076
Train Epoch: 18 Loss= 0.958640635  Accuracy= 0.8114
Train Epoch: 19 Loss= 0.936476767  Accuracy= 0.8138
Train Epoch:

In [1]:
# 测试集
accu_test =  sess.run(accuracy,
                      feed_dict={x: mnist.test.images, y: mnist.test.labels})

print("Test Accuracy:",accu_test)

# 验证集
accu_validation =  sess.run(accuracy,
                      feed_dict={x: mnist.validation.images, y: mnist.validation.labels})

print("Test Accuracy:",accu_validation)

# 训练集
accu_train =  sess.run(accuracy,
                      feed_dict={x: mnist.train.images, y: mnist.train.labels})

print("Test Accuracy:",accu_train)

NameError: name 'sess' is not defined

In [2]:
# 由于pred预测结果是one-hot编码格式，所以需要转换为0~9数字
prediction_result=sess.run(tf.argmax(pred,1), 
                           feed_dict={x: mnist.test.images })
                           
#查看预测结果中的前10项
prediction_result[0:10]

NameError: name 'sess' is not defined

In [3]:
import matplotlib.pyplot as plt
import numpy as np
def plot_images_labels_prediction(images,      # 图像列表
                                  labels,      # 标签列表
                                  prediction,  # 预测值列表
                                  index,       # 从第index个开始显示
                                  num=10 ):    # 缺省一次显示 10 幅
    fig = plt.gcf() # 获取当前图表，Get Current Figure
    fig.set_size_inches(10, 12)  # 1英寸等于 2.54 cm
    if num > 25: 
        num = 25            # 最多显示25个子图
    for i in range(0, num):
        ax = plt.subplot(5,5, i+1) # 获取当前要处理的子图
        
        ax.imshow(np.reshape(images[index],(28, 28)),  # 显示第index个图像
                  cmap='binary')
            
        title = "label=" + str(np.argmax(labels[index]))  # 构建该图上要显示的title信息
        if len(prediction)>0:
            title += ",predict=" + str(prediction[index]) 
            
        ax.set_title(title,fontsize=10)   # 显示图上的title信息
        ax.set_xticks([]);  # 不显示坐标轴
        ax.set_yticks([])        
        index += 1 
    plt.show()
    
    
plot_images_labels_prediction(mnist.test.images,
                              mnist.test.labels,
                              prediction_result,10,25)

NameError: name 'mnist' is not defined